In [1]:
%pylab
import scipy.integrate as integrate

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# solve_ivp functions

# returns vector of dx/dt, dAcx/dt and dAx/dt in that order

def derivs(t, y, nu, xAs):
    x0 = y[0]               # position of hub
    N = int(0.5*(len(y)-1))   # of lattice sites
    Acx = y[1:N+1]          # bound complex on lattice
    Ax = y[-N:]             # free complex on lattice
    
    dxcdt = integrate.trapz(-(x0-xAs)*Acx, x=xAs)  # force equation
    binding = exp(-0.5*(x0-xAs)**2)*Ax 
    burnt = nu*Acx

    dAcxdt = (binding - burnt)   # complex chemistry
    dAxdt = -binding            # free substrate chemistry
    
    return array( [dxcdt] + list(dAcxdt) + list(dAxdt))     # maybe a better way to put them all together

In [3]:
# define event as having gone to x = 20.0
def event(t, y):
    x0 = y[0]
        
    return 20.0-x0

event.direction = 0        
event.terminal = True     # stop solver if event satisfied

In [4]:
# use solve_ivp to find solution from t0=0 up to a max tf= 400, but stop if x hits x=20.0

# parameters - just 2 params in this model
tf = 400.0

a0_n = 10
nu_n = 15

a0s = logspace(-4, 3, a0_n, base=2)
nus = logspace(-5, 3, nu_n, base=2)

# define domain and lattice
Lmax = 30.0
Lmin = -10.0
dx = 0.1
Nx = int((Lmax-Lmin)/dx) +1
xAs = linspace(Lmin, Lmax, Nx)


In [15]:
# BDF

vs = []  # record final speed
burnt = []
effs = []
for a0 in a0s:
    
    A0x = zeros(Nx)
    A0x[xAs > 0.0] = a0
    
    vtmp = []
    btmp = []
    etmp = []
    for nu in nus:
        print(a0, nu)

        x0 = 0.0
        Ax = 1.0*A0x
        Acx = zeros(Nx)

        # initial conditions for all the equations, y0
        y0 = array([x0] + list(Acx) + list(Ax))

        sol = integrate.solve_ivp(lambda t, y: derivs(t, y, nu, xAs), (0., tf), y0, events=[event], method='BDF')

        speed = derivs(sol.t[-1], sol.y[:,-1], nu, xAs)[0]  #speed at final time
        vtmp.append(speed)
        
        Acx = sol.y[1:1+Nx]
        
        
        btot = trapz(nu*Acx[:,-1], x=xAs)    # total rate of burnt at final time
        btmp.append(btot)
        
        xp = sol.y[0][-1]
        xpindex = int((xp-Lmin)/dx)
        eff = trapz(Acx[xpindex:,-1], x = xAs[xpindex:])**2/btot    
        etmp.append(eff)
        
    vs.append(vtmp)
    burnt.append(btmp)
    effs.append(etmp)
vs = array(vs)
burnt = array(burnt)
effs = array(effs)

0.0625 0.03125
0.0625 0.04643732153552963
0.0625 0.06900559460461327
0.0625 0.10254191950095475
0.0625 0.15237670677555942
0.0625 0.2264309160659766
0.0625 0.336475048158089
0.0625 0.5
0.0625 0.7429971445684741
0.0625 1.104089513673812
0.0625 1.640670712015275
0.0625 2.4380273084089508
0.0625 3.6228946570556255
0.0625 5.383600770529422
0.0625 8.0
0.1071554978566341 0.03125
0.1071554978566341 0.04643732153552963
0.1071554978566341 0.06900559460461327
0.1071554978566341 0.10254191950095475
0.1071554978566341 0.15237670677555942
0.1071554978566341 0.2264309160659766
0.1071554978566341 0.336475048158089
0.1071554978566341 0.5
0.1071554978566341 0.7429971445684741
0.1071554978566341 1.104089513673812
0.1071554978566341 1.640670712015275
0.1071554978566341 2.4380273084089508
0.1071554978566341 3.6228946570556255
0.1071554978566341 5.383600770529422
0.1071554978566341 8.0
0.18371681153444983 0.03125
0.18371681153444983 0.04643732153552963
0.18371681153444983 0.06900559460461327
0.183716811534

In [45]:
# euler method

sol_a0 = []


for a0 in a0s:
    sol_nu = []
    for nu in nus: 
        print(a0, nu)
        tf = 400.0


        xp = 0.0
        As = zeros(Nx)
        As[xAs > 0.0] = a0
        Ac = zeros(Nx)

        dt = 0.001
        t_max = int(tf/dt)
        xp_t = []
        eff_t = []
        v_t = []
        burnt_t = []

        for t in range(t_max): 

            total_f = trapz(-(xp -xAs) * Ac, x=xAs)
            binding = exp(-0.5 * (xp - xAs) ** 2) * As
            burnt = nu * Ac
            Ac += (binding - burnt ) *dt
            As += (- binding) * dt
            xp += total_f * dt
            xp_t.append(xp)
            v_t.append(total_f)
            burnrate = trapz(Ac,x= xAs)*nu
            burnt_t.append(burnrate)
            xpindex = int((xp-Lmin)/dx)
            eff = trapz(-(xp -xAs[xpindex:]) * Ac[xpindex:], x=xAs[xpindex:])**2/burnrate
            eff_t.append(eff)


            if abs(xp) > 10:
                break

        sol_nu.append(eff_t[-1])
    sol_a0.append(sol_nu)
sol_a0 = array(sol_a0)

0.0625 0.03125
0.0625 0.04643732153552963
0.0625 0.06900559460461327
0.0625 0.10254191950095475
0.0625 0.15237670677555942
0.0625 0.2264309160659766
0.0625 0.336475048158089
0.0625 0.5
0.0625 0.7429971445684741
0.0625 1.104089513673812
0.0625 1.640670712015275
0.0625 2.4380273084089508
0.0625 3.6228946570556255
0.0625 5.383600770529422
0.0625 8.0
0.1071554978566341 0.03125
0.1071554978566341 0.04643732153552963
0.1071554978566341 0.06900559460461327
0.1071554978566341 0.10254191950095475
0.1071554978566341 0.15237670677555942
0.1071554978566341 0.2264309160659766
0.1071554978566341 0.336475048158089
0.1071554978566341 0.5
0.1071554978566341 0.7429971445684741
0.1071554978566341 1.104089513673812
0.1071554978566341 1.640670712015275
0.1071554978566341 2.4380273084089508
0.1071554978566341 3.6228946570556255
0.1071554978566341 5.383600770529422
0.1071554978566341 8.0
0.18371681153444983 0.03125
0.18371681153444983 0.04643732153552963
0.18371681153444983 0.06900559460461327
0.183716811534

In [ ]:
# plot v vs. nu
for i in range(0,a0_n):
    plot(nus, vs[i], '.-', label = r"$a_{tot}:$  " + str(round(a0s[i],2)))

title(r"$Steady\ State\ Velocity\ vs. \nu'$")
legend()
xlabel(r"$\nu'$")
ylabel("velocity")
xscale('log')

In [ ]:
# plot v vs. a0 
for i in range(0,nu_n,2):
    plot(a0s, vs.T[i], '.-', label = r"$\nu':$  " + str(round(nus[i],2)))

title(r"$Steady\ State\ Velocity\ vs. a_{tot}$")
legend()
xlabel(r"$a_{tot}$")
ylabel("velocity")
xscale('log')

In [ ]:
#plot burn rate vs. nu
for i in range(0,a0_n):
    plot(nus, burnt[i], '.-', label = r"$a_{tot}:$  " + str(round(a0s[i],2)))

title(r"$Burn\ Rate\ vs. \nu'$")
legend()
xlabel(r"$\nu'$")
ylabel("burn rate")
xscale('log')

In [ ]:
#plot burn rate vs. a0
for i in range(0,nu_n-6,2):
    plot(a0s, burnt.T[i], '.-', label = r"$\nu':$  " + str(round(nus[i],2)))

title(r"$Burn\ Rate\ vs. a_{tot}$")
legend()
xlabel(r"$a_{tot}$")
ylabel("burn rate")
xscale('log')

In [ ]:
#plot burn rate vs. a0
for i in range(0,nu_n-6,2):
    plot(a0s, burnt.T[i], '.-', label = r"$\nu':$  " + str(round(nus[i],2)))

title(r"$Burn\ Rate\ vs. a_{tot}$")
legend()
xlabel(r"$a_{tot}$")
ylabel("burn rate")
xscale('log')

In [58]:
#plot efficiency vs. a0
for i in range(0,nu_n,2):
    plot(a0s, sol_a0.T[i], '.-', label = r"$\nu'$:  " + str(round(nus[i],2)))

title(r"Relative Efficiency vs. $a_{tot}$")
legend()
xlabel(r"$a_{tot}$")
ylabel("efficiency")
#xscale('log')

Text(0, 0.5, 'efficiency')

In [59]:
#plot efficiency vs. nu
for i in range(0,a0_n,2):
    plot(nus, sol_a0[i], '.-', label = r"$a_{tot}:$:  " + str(round(a0s[i],2)))

title(r"Relative Efficiency vs. $\nu'$")
legend()
xlabel(r"$\nu'$")
ylabel("efficiency")
xscale('log')
#yscale("log")